<a href="https://colab.research.google.com/github/ttped/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/Trevor_Pedersen_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [131]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [132]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [133]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [134]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [135]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [136]:
df = df.set_index('SALE_DATE')
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE_DATE,,,,,,,,,,,,,,,,,,,,
01/01/2019,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0
01/01/2019,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0
01/01/2019,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0
01/01/2019,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0
01/01/2019,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0


In [137]:
df = df.sort_index()
df.head(3)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE_DATE,,,,,,,,,,,,,,,,,,,,
01/01/2019,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0
01/01/2019,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0
01/01/2019,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0


In [138]:
df.index = pd.to_datetime(df.index)

In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 23040 entries, 2019-01-01 to 2019-04-30
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         23040 non-null  object 
 1   NEIGHBORHOOD                    23040 non-null  object 
 2   BUILDING_CLASS_CATEGORY         23040 non-null  object 
 3   TAX_CLASS_AT_PRESENT            23039 non-null  object 
 4   BLOCK                           23040 non-null  int64  
 5   LOT                             23040 non-null  int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       23039 non-null  object 
 8   ADDRESS                         23040 non-null  object 
 9   APARTMENT_NUMBER                5201 non-null   object 
 10  ZIP_CODE                        23039 non-null  float64
 11  RESIDENTIAL_UNITS               23039 non-null  float64
 12  COMMERCIAL_UNIT

In [140]:
#Convert appropriate columns to numeric
df['LAND_SQUARE_FEET'] = pd.to_numeric(df['LAND_SQUARE_FEET'], errors='coerce')
df['LAND_SQUARE_FEET'].dtype

dtype('float64')

In [141]:
#Filter data
#Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.
df = df[(df['SALE_PRICE'] > 100000) & (df['SALE_PRICE'] < 2000000) & (df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS')]

In [245]:
#Feature Engineering
#Find average sale price of zip code
temp = df[['ZIP_CODE', 'SALE_PRICE']].groupby(by='ZIP_CODE').mean()
print(temp)
temp2 = df['ZIP_CODE'].map(lambda x: temp['SALE_PRICE'][temp.index == x].values[0])
print(temp2.values)
print(temp2.dtype)
df['ZIP_CODE_MEAN_SALE_PRICE'] = temp2.values
df[['ZIP_CODE_MEAN_SALE_PRICE','ZIP_CODE']].sort_values(by='ZIP_CODE')

            SALE_PRICE
ZIP_CODE              
10030.0   1.800000e+06
10301.0   6.323767e+05
10302.0   3.941374e+05
10303.0   3.813523e+05
10304.0   5.743424e+05
...                ...
11691.0   5.043359e+05
11692.0   3.088026e+05
11693.0   3.252500e+05
11694.0   8.422037e+05
11697.0   3.748111e+05

[125 rows x 1 columns]
[1003860.65517241  504886.52941176  545681.33333333 ...  532499.81428571
  449093.79        487073.77083333]
float64


,ZIP_CODE_MEAN_SALE_PRICE,ZIP_CODE
SALE_DATE,,
2019-04-16,1.800000e+06,10030.0
2019-02-26,6.323767e+05,10301.0
2019-02-22,6.323767e+05,10301.0
2019-02-22,6.323767e+05,10301.0
2019-02-15,6.323767e+05,10301.0
...,...,...
2019-04-01,3.748111e+05,11697.0
2019-01-16,3.748111e+05,11697.0
2019-01-16,3.748111e+05,11697.0


In [243]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,ZIP_CODE_MEAN_SALE_PRICE
SALE_DATE,,,,,,,,,,,,,,,,,,,,,
2019-01-01,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,NaN,1325.0,1930.0,1,A9,550000,1.003861e+06
2019-01-01,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,NaN,2001.0,1940.0,1,A1,200000,5.048865e+05
2019-01-02,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,NaN,2043.0,1925.0,1,A1,810000,5.456813e+05
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,NaN,2680.0,1899.0,1,A1,125000,7.583333e+05
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,NaN,1872.0,1940.0,1,A5,620000,5.393785e+05


In [264]:
import numpy as np
#Below is the crappy OneHotEncoder since it can't automatically select columns for OneHotEncoding
#from sklearn.preprocessing import OneHotEncoder
from category_encoders import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import mean_squared_error

In [297]:
#Model features a one-hot-encoder, Select K Best, and linear regression
def model(df, target, features_arr):
  X = df[features_arr]
  y = df[target]

  #Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
  mask = df.index.map(lambda x: True if x.month in range(1,4) and x.year == 2019 else False)
  invert_mask = mask.map(lambda x: not x)

  X_train = X[mask]
  y_train = y[mask]

  X_val = X[invert_mask]
  y_val = y[invert_mask]

  #Note: instead of cat names you can use cols=[''] to pass in exact columns to encode
  #Do one-hot encoding of categorical features.
  transformer = OneHotEncoder(use_cat_names=True)

  #Fits transformer to training data (I'm not totally sure what that means)
  transformer.fit(X_train)

  #Note: the one hot encoder is smart enough to find columns with 100% string values and encode them (I think 100% string to work)
  #One hot encodes categorical data
  XT_train = transformer.transform(X_train)
  XT_val = transformer.transform(X_val)
  
  #Do feature selection with SelectKBest.
  #Note: what is the best value for k?
  #num = round(len(features)/2)
  num = round(len(features)/1.5)
  transformer_2 = SelectKBest(k=num) #k is a hyperparameter

  #Fit transformer to training data
  transformer_2.fit(XT_train, y_train)

  #Transform the training data
  XTT_train = transformer_2.transform(XT_train)
  XTT_val = transformer_2.transform(XT_val)

  #Create the model
  model = LinearRegression()

  #Fit model to training data
  model.fit(XTT_train, y_train)

  LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

  #Training predictions derived from feature data
  y_pred_train = model.predict(XTT_train)

  #Validation predictions derived from feature data
  y_pred = model.predict(XTT_val)

  #Get mean absolute error for the test set.
  print("Training Mean absolute error:", mean_absolute_error(y_train, y_pred_train))
  print("Validation Mean absolute error:", mean_absolute_error(y_val, y_pred), '\n')

  print('Training MAE:', mean_absolute_error(y_train, y_pred_train))
  print('Validation MAE:', mean_absolute_error(y_val, y_pred), '\n')

  print("Traning RMSE:", mean_squared_error(y_train, y_pred_train, squared=False))
  print("Validation RMSE:", mean_squared_error(y_val, y_pred, squared=False), '\n')

  print("Training R-squared score is:", model.score(XTT_train, y_train))
  print("Validation R-squared is:", model.score(XTT_val, y_val), '\n')

In [246]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3151 entries, 2019-01-01 to 2019-04-30
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         3151 non-null   object 
 1   NEIGHBORHOOD                    3151 non-null   object 
 2   BUILDING_CLASS_CATEGORY         3151 non-null   object 
 3   TAX_CLASS_AT_PRESENT            3151 non-null   object 
 4   BLOCK                           3151 non-null   int64  
 5   LOT                             3151 non-null   int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       3151 non-null   object 
 8   ADDRESS                         3151 non-null   object 
 9   APARTMENT_NUMBER                1 non-null      object 
 10  ZIP_CODE                        3151 non-null   float64
 11  RESIDENTIAL_UNITS               3151 non-null   float64
 12  COMMERCIAL_UNITS

In [301]:
model(df, 'SALE_PRICE', ['NEIGHBORHOOD', 'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_CATEGORY', 'BUILDING_CLASS_AT_PRESENT', 'BOROUGH', 'ZIP_CODE_MEAN_SALE_PRICE'])

Training Mean absolute error: 145956.47311071018
Validation Mean absolute error: 145940.60711324433 

Training MAE: 145956.47311071018
Validation MAE: 145940.60711324433 

Traning RMSE: 207950.91065631082
Validation RMSE: 215018.7233502993 

Training R-squared score is: 0.49125710351030455
Validation R-squared is: 0.4749832240504933 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 9 21 22 27] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [252]:
print(df.columns)

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'ZIP_CODE_MEAN_SALE_PRICE'],
      dtype='object')


In [302]:
features = df.columns.map(lambda x: x if x not in ['SALE_PRICE', 'APARTMENT_NUMBER', 'EASE-MENT', 'LAND_SQUARE_FEET', 'ADDRESS', 'LOT', 'BLOCK'] else np.NaN)
features = features.dropna()
print(features)
model(df, 'SALE_PRICE', features)

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE',
       'BUILDING_CLASS_AT_TIME_OF_SALE', 'ZIP_CODE_MEAN_SALE_PRICE'],
      dtype='object')
Training Mean absolute error: 144703.71643142952
Validation Mean absolute error: 145376.74053304523 

Training MAE: 144703.71643142952
Validation MAE: 145376.74053304523 

Traning RMSE: 206737.02188567317
Validation RMSE: 214300.36102309867 

Training R-squared score is: 0.4971792206921081
Validation R-squared is: 0.4784854516482257 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 4 12 26 27 34 45] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [276]:
('BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'ZIP_CODE_MEAN_SALE_PRICE')

('BOROUGH',
 'NEIGHBORHOOD',
 'BUILDING_CLASS_CATEGORY',
 'TAX_CLASS_AT_PRESENT',
 'BLOCK',
 'LOT',
 'EASE-MENT',
 'BUILDING_CLASS_AT_PRESENT',
 'ADDRESS',
 'APARTMENT_NUMBER',
 'ZIP_CODE',
 'RESIDENTIAL_UNITS',
 'COMMERCIAL_UNITS',
 'TOTAL_UNITS',
 'LAND_SQUARE_FEET',
 'GROSS_SQUARE_FEET',
 'YEAR_BUILT',
 'TAX_CLASS_AT_TIME_OF_SALE',
 'BUILDING_CLASS_AT_TIME_OF_SALE',
 'SALE_PRICE',
 'ZIP_CODE_MEAN_SALE_PRICE')

In [311]:
features = ['YEAR_BUILT', 'ZIP_CODE_MEAN_SALE_PRICE', 'COMMERCIAL_UNITS', 'RESIDENTIAL_UNITS', 'BUILDING_CLASS_AT_PRESENT', 'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_CATEGORY', 
            'NEIGHBORHOOD', 'GROSS_SQUARE_FEET']
model(df, 'SALE_PRICE', features)

Training Mean absolute error: 144729.86252741233
Validation Mean absolute error: 145295.91117062763 

Training MAE: 144729.86252741233
Validation MAE: 145295.91117062763 

Traning RMSE: 206740.73378106754
Validation RMSE: 214151.74269868777 

Training R-squared score is: 0.49716116456579545
Validation R-squared is: 0.4792085464891017 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [15 16 19] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [328]:
from sklearn.linear_model import Ridge
#Model features a one-hot-encoder, Select K Best, and ridge regression
def model_ridge_regression(df, target, features_arr):
  X = df[features_arr]
  y = df[target]

  #Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
  mask = df.index.map(lambda x: True if x.month in range(1,4) and x.year == 2019 else False)
  invert_mask = mask.map(lambda x: not x)

  X_train = X[mask]
  y_train = y[mask]

  X_val = X[invert_mask]
  y_val = y[invert_mask]

  #Note: instead of cat names you can use cols=[''] to pass in exact columns to encode
  #Do one-hot encoding of categorical features.
  transformer = OneHotEncoder(use_cat_names=True)

  #Fits transformer to training data (I'm not totally sure what that means)
  transformer.fit(X_train)

  #Note: the one hot encoder is smart enough to find columns with 100% string values and encode them (I think 100% string to work)
  #One hot encodes categorical data
  XT_train = transformer.transform(X_train)
  XT_val = transformer.transform(X_val)
  
  #Do feature selection with SelectKBest.
  #Note: what is the best value for k?
  #num = round(len(features)/2)
  num = round(len(features)/2)
  transformer_2 = SelectKBest(k=num) #k is a hyperparameter

  #Fit transformer to training data
  transformer_2.fit(XT_train, y_train)

  #Transform the training data
  XTT_train = transformer_2.transform(XT_train)
  XTT_val = transformer_2.transform(XT_val)

  #Fit a ridge regression model with multiple features. Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)
  #Create the model
  model = Ridge(alpha=0.2)

  #Fit model to training data
  model.fit(XTT_train, y_train)

  Ridge(alpha=0.2, copy_X=True, fit_intercept=True, max_iter=None, normalize=False, random_state=None, solver='auto', tol=0.001)

  #Training predictions derived from feature data
  y_pred_train = model.predict(XTT_train)

  #Validation predictions derived from feature data
  y_pred = model.predict(XTT_val)

  #Get mean absolute error for the test set.
  print("Training Mean absolute error:", mean_absolute_error(y_train, y_pred_train))
  print("Validation Mean absolute error:", mean_absolute_error(y_val, y_pred), '\n')

  print('Training MAE:', mean_absolute_error(y_train, y_pred_train))
  print('Validation MAE:', mean_absolute_error(y_val, y_pred), '\n')

  print("Traning RMSE:", mean_squared_error(y_train, y_pred_train, squared=False))
  print("Validation RMSE:", mean_squared_error(y_val, y_pred, squared=False), '\n')

  print("Training R-squared score is:", model.score(XTT_train, y_train))
  print("Validation R-squared is:", model.score(XTT_val, y_val), '\n')


In [329]:
features = ['YEAR_BUILT', 'ZIP_CODE_MEAN_SALE_PRICE', 'COMMERCIAL_UNITS', 'RESIDENTIAL_UNITS', 'BUILDING_CLASS_AT_PRESENT', 'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_CATEGORY', 
            'NEIGHBORHOOD', 'GROSS_SQUARE_FEET']
model_ridge_regression(df, 'SALE_PRICE', features)

Training Mean absolute error: 148921.32165091875
Validation Mean absolute error: 148011.6489905355 

Training MAE: 148921.32165091875
Validation MAE: 148011.6489905355 

Traning RMSE: 215210.6451663489
Validation RMSE: 218221.0350221747 

Training R-squared score is: 0.45511581357340525
Validation R-squared is: 0.4592284374127493 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [15 16 19] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
